# Important Imports

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.circuit.library import QFT
from math import gcd, ceil, log2
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
import random  

# Getting Started

#### 1. Number to Factorize (`N`)

The number `N` is the integer that we want to factorize. This value is chosen as an input to the quantum algorithm. For this example, `N` is set to `15`, but in practice, it can be any positive integer that needs factorization.

#### 2. Number of Bits (`n`)

The number of bits `n` represents how many bits are needed to represent the number `N` in binary form. This is computed by taking the ceiling of the logarithm (base 2) of `N`, which ensures that the number of bits is sufficient to represent the value of `N`. For example, `15` requires 4 bits (`n = 4`), as `np.ceil(np.log2(15))` results in `4`.

#### 3. Quantum Registers

Quantum registers are used to store quantum bits (qubits) during quantum operations. The setup involves two main quantum registers:

- **Input Quantum Register (`q`)**: This register holds the qubits that represent the input quantum state. The size of this register is `2n`, which is chosen to accommodate both the quantum state and the operations that will be performed during the algorithm. The factor of 2 comes from the typical structure of many quantum algorithms, where two registers are used for the input and output states.

- **Target Quantum Register (`t`)**: This register contains `n` qubits, which are used for the target state in quantum operations. It stores intermediate and final quantum states during the computation.

#### 4. Classical Register (`c`)

After quantum operations are performed, the final results need to be measured and stored. A classical register is used for this purpose. The classical register holds `2n` classical bits that store the measurement results of the quantum states. These results are obtained after performing quantum measurements on the quantum registers, and they are essential for interpreting the outcome of the quantum computation.

### Selecting a Base `a` for Quantum Factorization

This process selects a random integer `a` that is coprime with a number `N` to be used in a quantum factoring algorithm. The base `a` must satisfy the condition that `gcd(a, N) = 1`.

#### Steps:

1. **Randomly select `a`**: Choose `a` from the range `[2, N)`.
2. **Check coprimeness**: If `gcd(a, N) > 1`, choose a new `a`.
3. **Repeat** until a valid `a` is found.
4. **Output** the chosen base `a`.

This ensures that `a` and `N` are coprime, which is essential for algorithms like Shor's algorithm to work correctly.


### Building Circuit

#### `Uf` Function for Quantum Factorization

The `Uf` function defines a quantum operation for modular exponentiation, which is used in algorithms like Shor's algorithm. This function applies a series of controlled-phase gates based on the powers of `a` modulo `N`.

##### Function Overview

- **Purpose**: The function applies modular exponentiation to the quantum register, which is a core operation for quantum factoring algorithms.
- **Input**: A quantum circuit `circ` which has the necessary quantum registers (`q` and `t`) set up.
- **Operation**: It iterates over the range `2*n` (where `n` is the number of bits representing `N`) and applies a controlled-phase gate to the quantum register for each power of `a`, computed modulo `N`.

##### Steps in the Function

1. **Modular Exponentiation**: For each iteration, it computes `a^(2^i) % N` using `pow(a, 2**i, N)`.
2. **Controlled-Phase Gates**: The function applies a controlled-phase gate to the quantum register. The phase is determined by `2 * np.pi * mod_exp / N`, where `mod_exp` is the result of the modular exponentiation.
3. **Iteration**: This process repeats for `i` in the range `[0, 2*n)`.

##### Key Concepts

- **Controlled-Phase Gate** (`cp`): A gate that applies a phase shift based on the control qubit.
- **Modular Exponentiation**: A fundamental operation in Shor's algorithm that is used to find periodicity in quantum systems.


### Quantum Phase Estimation (QPE) Circuit

This document outlines the steps involved in constructing a **Quantum Phase Estimation (QPE)** circuit. The QPE algorithm is used for estimating the eigenvalue (or phase) of a unitary operator. It is an essential component in many quantum algorithms, such as **Shor's algorithm** for factoring large numbers.

#### QPE Circuit Breakdown

##### 1. **Initialization**
The circuit begins by initializing the quantum registers:
- `q`: The input qubits (quantum register).
- `t`: The target qubits (quantum register).
- `c`: The classical register.

The initialization steps are as follows:
- `qpe.x(t)`: The target qubits are initialized to the state \( \lvert 1 \rangle \).
- `qpe.h(q)`: A Hadamard gate is applied to all qubits in the input register `q` to create an equal superposition of all possible states.

##### 2. **Modular Exponentiation**
The core of the QPE circuit involves applying the modular exponentiation, which encodes the phase information in the target register `t`. This step is performed by the `Uf` function, which applies modular exponentiation based on the chosen base `a` and the number `N` to be factored.

##### 3. **Quantum Fourier Transform (QFT)**
After applying modular exponentiation, the quantum state in the input register needs to be transformed to extract the phase. This is achieved by applying the **Quantum Fourier Transform (QFT)**:
- `qpe.append(QFT(2*n, inverse=True), q)`: The inverse QFT is applied to the input qubits in the register `q` to convert the phase information into a measurable format.

##### 4. **Measurement**
Finally, the quantum state is measured:
- `qpe.measure(range(n*2), range(n*2))`: This command measures all qubits and stores the results in the classical register `c`. The measurement collapses the quantum state, extracting the eigenvalue (phase) from the quantum state.

##### 5. **Drawing the Circuit**
- `qpe.draw()`: This command visually represents the quantum circuit, showing all qubits, gates, and measurements in the QPE process.

#### Purpose of QPE in Quantum Algorithms

In algorithms like Shor’s, QPE is used to estimate the phase of a modular exponentiation operator, which is essential for determining the periodicity of a function. The phase information is then used to identify non-trivial factors of a number, enabling quantum algorithms to solve problems that are difficult for classical computers.

#### Summary

The QPE circuit applies several key quantum operations:
- Hadamard gates to create superposition,
- Modular exponentiation to encode phase information,
- Quantum Fourier Transform to extract the phase,
- Measurement to obtain classical output.

This setup is central to algorithms that require phase estimation, such as Shor's algorithm for factoring large numbers.


# Testing Everything

In [ ]:
import os
token = os.environ.get("QUANTUM_TOKEN")
instance=os.environ.get("QUANTUM_INSTANCE")
service = QiskitRuntimeService(channel="ibm_cloud", token=token, instance=instance)

### Modular Exponentiation

In [5]:
def Uf(circ):
    for i in range(2*n):
        base = int(a)
        exp = int(2**i)
        mod = int(N)
        mod_exp = pow(base, exp, mod)
        circ.cp(2*np.pi*float(mod_exp)/float(N), q[i], t)

### Retrievin Denominator

In [3]:
from fractions import Fraction

def denominator(decimal_value, n, N):
    return (Fraction(decimal_value / 2**(n*2)).limit_denominator(N)).denominator


def retrieve_denominator(decimal_value, n, N):
    try:
        # Conversione e validazione input
        decimal_value = int(decimal_value)
        n = int(n)
        N = int(N)
        
        # Calcolo diretto, senza HTTP
        return denominator(decimal_value, n, N)
    
    except Exception as e:
        return f"Error: {str(e)}"


In [ ]:
N = 851 # number to factorize
N = int(N)
n = ceil(log2(N))  # number of bits to represent N


q = QuantumRegister(2*n, "q")
t = QuantumRegister(n, "t")
c = ClassicalRegister(2*n, "c")

a = random.randint(2, N-1)

while gcd(a, N) > 1:
    a = random.randint(2, N-1)

print(f"Chosen base a = {a}")


qpe = QuantumCircuit(q, t, c)
qpe.x(t)
qpe.barrier()
qpe.h(q)
Uf(qpe)
qpe.append(QFT(2*n, inverse=True), q)
qpe.measure(range(n*2), range(n*2))
qpe.barrier()
qpe.measure(q, c)
backend = service.backend("ibm_fez")


compiled = transpile(qpe, backend)
sampler = SamplerV2(mode=backend)
job = sampler.run([compiled], shots=1024)
result = job.result()
counts = result[0].data.c.get_counts()
sorted_measurements = sorted(counts.items(), key=lambda x: x[1], reverse=True)
j=0
for measurement, _ in sorted_measurements:
    j+=1
    decimal_value = int(measurement, 2)
    r = retrieve_denominator(decimal_value, n, N)
    
    if isinstance(r, int) and r % 2 == 0:
        x = pow(a, r // 2, N)
        f1 = gcd(x - 1, N)
        f2 = gcd(x + 1, N)
        
        if 1 < f1 < N:
            if N % f1 == 0:
                print(f"The factors for {N} are: {f1}, {N // f1}")
                break

        if 1 < f2 < N:
            if N % f2 == 0:
                print(f"The factors for {N} are: {f2}, {N // f2}")
                break
                

Chosen base a = 178
The factors for 851 are: 23, 37


In [10]:
print(sorted_measurements)

[('01010100111110010001', 1), ('10110001001000111000', 1), ('10010011000000001101', 1), ('01000111111001011000', 1), ('10110110110010011110', 1), ('11100110000100101000', 1), ('11100101111000100111', 1), ('01001100010101010111', 1), ('01111111100101111000', 1), ('11000000011111001010', 1), ('11110110010001100100', 1), ('01110011101110100100', 1), ('01011011000110110000', 1), ('11100001011000011010', 1), ('10100001000011110000', 1), ('00000000001010000101', 1), ('10100000111111001000', 1), ('01111101111001111100', 1), ('01110100001000100111', 1), ('00111111101001101000', 1), ('00101001010100000011', 1), ('11100001110111100010', 1), ('00011010000010110011', 1), ('11011100011001110100', 1), ('00000001101101111110', 1), ('00001000100001001000', 1), ('01000101011100000000', 1), ('01011101001110101000', 1), ('01000001111001010100', 1), ('11000001011101010000', 1), ('00011111000010000011', 1), ('01010111000011001000', 1), ('11100010001001110101', 1), ('11111001110010100001', 1), ('11111000100

The measurement results show a flat distribution, with each bitstring appearing only once.  
This indicates that the quantum circuit did not produce clear interference patterns in the phase register.  
In the context of Shor’s algorithm, this means that the Quantum Phase Estimation step failed to extract the periodicity of \( a^x \bmod N \).  

Such behavior is expected when running on real quantum hardware for large \( N \) (in this case, \( N = 851 \)), because the circuit depth—especially due to the controlled modular exponentiation and the inverse QFT—makes the algorithm highly sensitive to noise and decoherence.  

As a result, the measurement outcomes are effectively random, preventing the recovery of the correct period \( r \).  
This experiment still validates the logical structure of Shor’s algorithm, even though current hardware cannot yet execute it reliably at this scale.


In [9]:
print(f"  - Profondità dopo transpilazione: {compiled.depth()}")
print(f"  - Gates dopo transpilazione: {len(compiled.data)}")

  - Profondità dopo transpilazione: 2018
  - Gates dopo transpilazione: 7820
